In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import cv2
import numpy as np

import os
import subprocess
from pathlib import Path

from maddrive_adas.sign_det.base import AbstractSignClassifier
from maddrive_adas.sign_det.classifier import EncoderBasedClassifier
from maddrive_adas.utils.fs import imread_rgb

try:
    PROJECT_ROOT = Path(os.readlink(f'/proc/{os.environ["JPY_PARENT_PID"]}/cwd'))
except FileNotFoundError:
    __file = %pwd
    PROJECT_ROOT = Path(__file).parents[1]
    
data_path: Path = Path(PROJECT_ROOT / 'SignDetectorAndClassifier/data/additional_sign')
CLASSIFIER_ARCHIVE = PROJECT_ROOT / 'encoder_archive'
SUBCLASSIFIER_ARCHIVE = PROJECT_ROOT / 'SignDetectorAndClassifier/data/classifier_chpt_07.03_00.07_TRAIN_ACC0.9231_VALID_ACC0.9091_EPOCH127'
c: AbstractSignClassifier = EncoderBasedClassifier(
    config_path=str(CLASSIFIER_ARCHIVE),
    path_to_subclassifier_3_24_and_3_25_config=str(SUBCLASSIFIER_ARCHIVE)
)

In [ ]:
wrongs = 0
total = 0
for file in data_path.iterdir():
    if '3.24' in str(file) or '3.25' in str(file):
        total += 1
        img = imread_rgb(file)
        res = c.classify(img)[1][0]
        predicted_sign = res[0]
        conf = res[1]
        # print(res)
        sign_class = file.stem.split('_')[0].rsplit('.', 1)[0]
        sign_class_verbose = file.stem.split('_')[0]
        
        mark_as_bad = sign_class_verbose != res[0]

        fig, ax = plt.subplots(1, 1, figsize=(4,4))
        
        ax.title.set_text(f'{predicted_sign}:{conf}')
        ax.imshow(img)
        if mark_as_bad:
            wrongs += 1
            ax.patch.set_edgecolor('red') 
            ax.patch.set_linewidth('30')
        # break
print(1 - wrongs / total)        
plt.show()